# Import des libraries

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2

ERROR: Could not find a version that satisfies the requirement torch==1.5.1 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.5.1


In [ ]:
import shutil
shutil.rmtree('/content/Deformable-DETR', ignore_errors=True)

In [ ]:
!git clone https://github.com/fundamentalvision/Deformable-DETR.git
%cd /content/Deformable-DETR
!pip install -q -r requirements.txt --upgrade

Cloning into 'Deformable-DETR'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 98 (delta 27), reused 25 (delta 25), pack-reused 37 (from 1)
Receiving objects: 100% (98/98), 383.50 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/Deformable-DETR


In [ ]:
import sys
sys.path.append('/content/Deformable-DETR')

In [ ]:
!pip install pycocotools

# Importation

In [ ]:
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as T
from pycocotools.coco import COCO
from torchvision.datasets import CocoDetection
from util.misc import nested_tensor_from_tensor_list
from models.deformable_detr import build_deformable_detr
import numpy as np
import random


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: cannot import name '_NewEmptyTensorOp' from 'torchvision.ops.misc' (/usr/local/lib/python3.11/dist-packages/torchvision/ops/misc.py)

# Préparation du dataset

In [ ]:
!curl -L https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip -o coco128.zip
!unzip -q coco128.zip

In [ ]:
import os
import json
from PIL import Image
from pathlib import Path

# Dossiers d'entrée
images_dir = Path("/content/coco128/images/train2017")
labels_dir = Path("/content/coco128/labels/train2017")

# Dictionnaire COCO de sortie
coco_dict = {
    "images": [],
    "annotations": [],
    "categories": []
}

# Mapping YOLO class_id → label name
# Tu peux adapter cela avec les vrais noms si tu les as
coco_class_names = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush"
]
  # À adapter
for idx, name in enumerate(coco_class_names):
    coco_dict["categories"].append({
        "id": idx,
        "name": name,
        "supercategory": "object"
    })

annotation_id = 1  # ID global pour chaque annotation

for image_id, image_path in enumerate(sorted(images_dir.glob("*.jpg"))):
    # Charger l'image pour obtenir ses dimensions
    img = Image.open(image_path)
    width, height = img.size

    # Ajouter info image
    coco_dict["images"].append({
        "id": image_id,
        "file_name": image_path.name,
        "width": width,
        "height": height
    })

    # Correspondant .txt
    label_path = labels_dir / f"{image_path.stem}.txt"
    if not label_path.exists():
        continue

    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, w, h = map(float, parts)
            class_id = int(class_id)

            # Convertir en format COCO (bbox = [x_min, y_min, width, height])
            x_min = (x_center - w / 2) * width
            y_min = (y_center - h / 2) * height
            bbox_width = w * width
            bbox_height = h * height

            coco_dict["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "bbox": [x_min, y_min, bbox_width, bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            })
            annotation_id += 1

# Sauvegarder le fichier COCO
output_path = "/content/coco128/annotations/instances_train2017.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(coco_dict, f, indent=2)

print(f"Annotations COCO sauvegardées dans {output_path}")

In [ ]:
class CocoWrapper(torch.utils.data.Dataset):
    def __init__(self, coco_dataset):
        self.coco_dataset = coco_dataset

    def __getitem__(self, idx):
        img, targets = self.coco_dataset[idx]
        boxes = []
        labels = []
        for target in targets:
            if 'bbox' in target:
                x, y, w, h = target['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(target['category_id'])
        if len(boxes) == 0:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)
        target_out = {'boxes': boxes, 'labels': labels}
        return img, target_out

    def __len__(self):
        return len(self.coco_dataset)


In [ ]:
transform = T.Compose([T.ToTensor()])
coco_dataset = CocoDetection(
    root='/content/coco128/images/train2017',
    annFile='/content/coco128/annotations/instances_train2017.json',
    transform=transform
)

wrapped_dataset = CocoWrapper(coco_dataset)
indices = torch.randperm(len(wrapped_dataset)).tolist()
train_dataset = Subset(wrapped_dataset, indices[:-50])
valid_dataset = Subset(wrapped_dataset, indices[-50:])

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)


# Mise en place du modèle

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Création du modèle
model, criterion, postprocessors = build_deformable_detr(num_classes=91,device=device)  # COCO has 91 classes

# Envoi du modèle sur le device

model.to(device)

# Optimiseur
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


# Train

In [ ]:
param_dicts = [
    {
        "params": [p for n, p in model.named_parameters() if "backbone" not in n and p.requires_grad]
    },
    {
        "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
        "lr": 1e-5,
    },
]

optimizer = torch.optim.AdamW(param_dicts, lr=1e-4, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
for epoch in range(5):
    train_one_epoch(model, criterion, train_loader, optimizer, device, epoch, print_freq=100)
    lr_scheduler.step()
    evaluate(model, criterion, postprocessors, val_loader, device=device)

print("\n✅ Entraînement terminé")

In [ ]:
model.train()
for epoch in range(2):
    running_loss = 0.0
    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [
            {k: v.to(device) for k, v in t.items()} for t in targets
        ]

        # Forward
        outputs = model(images)
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        # Backprop
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        running_loss += losses.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

print("\n✅ Entraînement terminé")